In [17]:
import os
import numpy as np
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt

In [18]:
%matplotlib inline

In [19]:
sen_raw = []
no_mow_raw = {"datetime": [], "text": []}
rain_raw = {"datetime": [], "text": []}
randy_raw = {"datetime": [], "cat": [], "on": []}

for fn in os.listdir():
    if not fn.endswith(".log"):
        continue
    
    print("parsing", fn)
    with open(fn, "r") as f:
        lines = f.readlines()
            
    state = "searching"
    for lineno, line in enumerate(lines):
        try:
            if state == "searching":
                sensor_sample = []
                if "[INFO] JKey.LOCAL_SENSOR" in line:
                    sensor_sample.append(dt.datetime.strptime(line[:19], "%Y-%m-%d %H:%M:%S"))
                    state = "sensor_1"
                elif "No mowing" in line:
                    no_mow_raw["datetime"].append(dt.datetime.strptime(line[:19], "%Y-%m-%d %H:%M:%S"))
                    no_mow_raw["text"].append(line.split("]")[1].strip())
                elif "raining" in line:
                    rain_raw["datetime"].append(dt.datetime.strptime(line[:19], "%Y-%m-%d %H:%M:%S"))
                    rain_raw["text"].append(line.split("]")[1].strip())
                elif "[INFO] JKey.SWITCH_RANDY" in line:
                    randy_raw["datetime"].append(dt.datetime.strptime(line[:19], "%Y-%m-%d %H:%M:%S"))
                    randy_raw["cat"].append(line.split("'")[1])
                    randy_raw["on"].append(1 if "True" in line else 0)
            elif state == "sensor_1":
                sensor_sample.append(float(line[7:11]))  # T_CPU
                state = "sensor_2"
            elif state == "sensor_2":
                _, temp, hum = line.split("=")
                sensor_sample.append(float(temp[:4]))  # T_int
                sensor_sample.append(float(hum[:4])) # H_int
                # plausibility check
                hum = float(hum[:4])
                if hum > 100:
                    raise ValueError("humidity={}".format(hum))
                state = "sensor_3"
            elif state == "sensor_3":
                _, temp, hum = line.split("=")
                sensor_sample.append(float(temp[:4]))  # T_ext
                sensor_sample.append(float(hum[:4])) # H_ext
                # plausibility check
                hum = float(hum[:4])
                if hum > 100:
                    raise ValueError("humidity={}".format(hum))
                sen_raw.append(sensor_sample)
                state = "searching"
        except Exception as e:
            print(e)
            print("\tfilename", fn, "lineno", lineno)
            print("\tline", line)
            state = "searching"
            
            

parsing raspi_garden_2019-05.log
parsing raspi_garden_2019-07.log
parsing raspi_garden_2019-08.log
parsing raspi_garden_2020-04.log
parsing raspi_garden_2020-06.log
parsing raspi_garden_2020-08.log
parsing raspi_garden_2021-10.log
parsing raspi_garden_2022-03.log
parsing raspi_garden_2022-04.log
parsing raspi_garden_2022-05.log


# Eval sprinkler rain_raws

In [20]:
rain = pd.DataFrame.from_records(rain_raw, columns=rain_raw.keys(), index="datetime")
rain.tail()

,text
datetime,
2021-06-14 21:21:54,manual raining initiated
2021-06-17 21:36:14,manual raining initiated
2021-06-21 19:10:38,manual raining initiated
2022-03-12 15:45:37,manual raining initiated
2022-05-12 16:43:20,manual raining initiated


# Eval no mowings

In [21]:
no_mow = pd.DataFrame.from_records(no_mow_raw, index="datetime")
no_mow

,text
datetime,
2019-05-21 08:00:00,No mowing because weatherForecast is None!
2019-05-22 08:00:00,No mowing because of rain: 0.19
2019-05-23 08:00:00,No mowing because weatherForecast is None!
2019-05-25 09:00:00,No mowing because of rain: 0.66
2019-05-27 09:00:00,No mowing because of rain: 0.10
...,...
2022-04-26 09:00:00,No mowing because of rain: 0.26
2022-05-06 09:00:00,No mowing because of rain: 0.31
2022-05-07 09:00:00,No mowing because of rain: 0.16


In [22]:
no_mow.text.value_counts()

No mowing because of rain: 0.10    6
No mowing because of rain: 0.31    6
No mowing because of rain: 0.19    6
No mowing because of rain: 0.17    5
No mowing because of rain: 0.41    5
                                  ..
No mowing because of rain: 0.43    1
No mowing because of rain: 0.28    1
No mowing because of rain: 0.95    1
No mowing because of rain: 0.55    1
No mowing because of rain: 1.04    1
Name: text, Length: 63, dtype: int64

# Eval switch randy

In [23]:
randy = pd.DataFrame.from_records(randy_raw, columns=randy_raw.keys(), index="datetime")
randy.tail()

,cat,on
datetime,,
2022-05-14 18:00:00,mow,0
2022-05-15 18:00:00,mow,0
2022-05-16 18:00:00,mow,0
2022-05-17 09:00:00,mow,1
2022-05-17 18:00:00,mow,0


In [24]:
randy.cat.value_counts()

mow    935
Name: cat, dtype: int64

In [25]:
%matplotlib qt

In [26]:
plt.figure(figsize=(14, 2))
plt.step(randy.index, randy["on"], label="switch randy")
plt.plot(no_mow.index, np.ones_like(list(no_mow.index))*0.5, "v", label="OWM no mow")
plt.legend();

# Eval sensors

In [27]:
columns=["datetime", "T_CPU", "T_int", "H_int", "T_ext", "H_ext"]
sen = pd.DataFrame.from_records(sen_raw, columns=columns, index="datetime")
sen.tail()

,T_CPU,T_int,H_int,T_ext,H_ext
datetime,,,,,
2022-05-17 16:40:31,69.8,47.0,21.4,33.3,20.2
2022-05-17 17:40:31,68.8,45.6,21.8,32.1,19.0
2022-05-17 18:40:33,61.2,45.0,22.1,26.8,25.2
2022-05-17 19:40:31,58.0,37.5,24.5,26.8,25.2
2022-05-17 20:40:31,54.8,34.2,25.5,24.6,29.0


In [28]:
sen[["T_ext", "T_int"]].plot(style=".", grid="both", figsize=(14, 4));

In [29]:
sen[["H_ext", "H_int"]].plot(style=".", grid="both", figsize=(14, 4));